In [4]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 MB 47.4 MB/s  0:00:02 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for opencv-python: filename=opencv_python-4.12.0.88-cp39-cp39-macosx_10_16_x86_64.whl size=28129518 sha256=b209703017a303ef9fda59e0e04d3831d7b1eafce60bbdf7439ea60c285f2443
  Stored in directory: /private/var/folders/81/1f5vsc1j7_dcjxl97qfpwqy00000gp/T/pip-ephem-wheel-cache-nm_9b2se/wheels/fa/14/4e/a36eecda2d43fe1f908012cd2a21484e602714995b785b12d6
Successfully built opencv-python


In [6]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [matplotlib]8 [matplotlib]


In [2]:
import cv2, glob, os

folder = "images"   # change if needed
paths = sorted(glob.glob(os.path.join(folder, "*.jpg")))

if not paths:
    raise FileNotFoundError("No .jpg files found in folder")

sizes = []
for p in paths:
    img = cv2.imread(p)
    if img is None:
        print("Warning: could not read", p)
        continue
    h, w = img.shape[:2]
    sizes.append((os.path.basename(p), w, h))

# Print basic stats
max_w = max(s[1] for s in sizes)
max_h = max(s[2] for s in sizes)
min_w = min(s[1] for s in sizes)
min_h = min(s[2] for s in sizes)

print(f"Total images read: {len(sizes)}")
print(f"Max width:  {max_w}")
print(f"Max height: {max_h}")
print(f"Min width:  {min_w}")
print(f"Min height: {min_h}")

# If there are different sizes, list them
unique_sizes = {}
for name, w, h in sizes:
    unique_sizes.setdefault((w, h), []).append(name)

print("\nUnique (width, height) combinations:")
for (w, h), names in unique_sizes.items():
    print(f"  {w}x{h}  -> {len(names)} images (e.g., {names[0]})")


Total images read: 72
Max width:  960
Max height: 720
Min width:  960
Min height: 720

Unique (width, height) combinations:
  960x720  -> 72 images (e.g., 11_4_2_000001_jpg.rf.bf209c05402c01422a9b5e875a8c4c66.jpg)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# --- EDIT THIS PART: your zone polygon points (in pixels) ---
zone_points = [ # x, y
    (500, 720), # bottom left
    (960, 720), # bottom right
    (960, 200), # top right
    (840, 180) # top left
]

# Image you want to test
img_path = "images/11_4_2_000001_jpg.rf.bf209c05402c01422a9b5e875a8c4c66.jpg"   # change if needed

# Load image
img = cv2.imread(img_path)
if img is None:
    raise FileNotFoundError("Image not found. Check img_path.")

# Convert BGR (OpenCV) → RGB (matplotlib)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Draw polygon
pts = np.array(zone_points, dtype=np.int32)
cv2.polylines(img_rgb, [pts], isClosed=True, color=(255,0,0), thickness=3)

# Show the result
plt.figure(figsize=(10, 8))
plt.imshow(img_rgb)
plt.title("Zone Polygon Test")
plt.axis("off")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# --- EDIT THIS PART: your zone polygon points (in pixels) ---
zone_points = [ # x, y
    (110, 520), # bottom left
    (220, 650), # bottom right
    (460, 360), # top right
    (370, 340) # top left
]

# Image you want to test
img_path = "images/11_4_2_000001_jpg.rf.bf209c05402c01422a9b5e875a8c4c66.jpg"   # change if needed

# Load image
img = cv2.imread(img_path)
if img is None:
    raise FileNotFoundError("Image not found. Check img_path.")

# Convert BGR (OpenCV) → RGB (matplotlib)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Draw polygon
pts = np.array(zone_points, dtype=np.int32)
cv2.polylines(img_rgb, [pts], isClosed=True, color=(255,0,0), thickness=3)

# Show the result
plt.figure(figsize=(10, 8))
plt.imshow(img_rgb)
plt.title("Zone Polygon Test")
plt.axis("off")
plt.show()

### Testing

In [14]:
import json

def load_zones(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    zones = []
    for z in data:
        zones.append(
            {
                "name": z["name"],
                "id": z.get("id", z["name"]),
                "points": [(float(x), float(y)) for x, y in z["points"]]
            }
        )
    return zones

def point_in_poly(x, y, poly):
    inside = False
    n = len(poly)
    for i in range(n):
        x1, y1 = poly[i]
        x2, y2 = poly[(i + 1) % n]
        cond = ((y1 > y) != (y2 > y))
        if cond:
            x_int = x1 + (y - y1) * (x2 - x1) / (y2 - y1)
            if x_int > x:
                inside = not inside
    return inside

def assign_zone_to_point(x, y, zones):
    for z in zones:
        if point_in_poly(x, y, z["points"]):
            return z["name"], z["id"]
    return "None", "None"

In [15]:
import os, glob, json, cv2, numpy as np

images_dir = "images"              # folder with images
json_dir   = "json"                # folder with per-image JSONs
zones_path = "zones_camera.json"   # your zone polygons

# Load zones once
zones = load_zones(zones_path)

# Get all json files
json_paths = sorted(glob.glob(os.path.join(json_dir, "*.json")))
print(f"Found {len(json_paths)} json files.")

# Output directory
out_dir = "debug_outputs_trial2"
os.makedirs(out_dir, exist_ok=True)

for jp in json_paths:
    # Load detections for this image
    with open(jp, "r") as f:
        dets = json.load(f)

    if not dets:
        print("Warning: empty detections in", jp)
        continue

    # Filter for class_id == 6
    dets = [d for d in dets if d["class_id"] == 6]

    # If everything filtered out, skip
    if len(dets) == 0:
        print("No class_id = 6 in", jp)
        continue

    # All entries share same image_name
    image_name = dets[0]["image_name"]
    img_path = os.path.join(images_dir, image_name)

    img = cv2.imread(img_path)
    if img is None:
        print("Warning: could not read image:", img_path)
        continue

    H, W = img.shape[:2]

    points = []   # bottom-center points
    labels = []   # zone IDs
    bboxes = []   # bbox pixel coords

    # Process each detection in this JSON
    for d in dets:
        xc = d["x_center"] * W
        yc = d["y_center"] * H
        w  = d["width"]  * W
        h  = d["height"] * H

        x1 = xc - w / 2
        x2 = xc + w / 2
        y1 = yc - h / 2
        y2 = yc + h / 2

        # bottom-center point
        u = xc
        v = y2

        # zone assignment (image-space polygons)
        zname, zid = assign_zone_to_point(u, v, zones)

        points.append((int(u), int(v)))
        labels.append(zid)
        bboxes.append((int(x1), int(y1), int(x2), int(y2)))

    # Draw zone polygons (red)
    for z in zones:
        pts = np.array(z["points"], dtype=np.int32)
        cv2.polylines(img, [pts], isClosed=True, color=(0, 0, 255), thickness=2)

    # Draw detection bboxes + bottom-center + zone id
    for (x1, y1, x2, y2), (u, v), zid in zip(bboxes, points, labels):
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        cv2.circle(img, (u, v), 5, (0, 255, 0), -1)
        cv2.putText(
            img, zid, (u + 5, v - 5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1
        )

    # Save debug image
    out_path = os.path.join(out_dir, f"debug_{image_name}")
    cv2.imwrite(out_path, img)
    print("Saved:", out_path)

print("Done.")

Found 72 json files.
Saved: debug_outputs_trial2/debug_11_4_2_000001_jpg.rf.bf209c05402c01422a9b5e875a8c4c66.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000002_jpg.rf.d7e5ce07f7e46c2a46c6a9180c4deaaa.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000003_jpg.rf.5fd50481c2918e5198bb9c689abecab8.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000004_jpg.rf.657998ccabf5d72ea94df50d0d2ee6eb.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000005_jpg.rf.05dd130b6d990a2cd24718bb7a8be41b.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000006_jpg.rf.d7690fe6a725b6a6ab2b3b2c914a85ee.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000007_jpg.rf.18075b9c19b6ed1eb060cf3455190cce.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000008_jpg.rf.f85db8247f17a0707256bc4769640cf6.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000009_jpg.rf.1cb10962e5705294b658d7721a961e41.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000010_jpg.rf.29591f8ac6fd8a8a8dd8886f574d69d6.jpg
Saved: debug_outputs_trial2/debug_11_4_2_000011_jpg.rf.3c90